In [1]:
import csv
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
df = pd.read_csv("../../data/data9size20.csv")
y = df['class']

In [2]:
img_list = []
size = 64
for i in range(len(df)):
    img_list.append(cv2.resize(cv2.imread("../../"+df.PATH[i]),(size,size)))
    
X_train, X_test, y_train, y_test = train_test_split(img_list, y, test_size=0.2,random_state=1)
X_test,X_val,y_test,y_val = train_test_split(X_test, y_test, test_size=0.5,random_state=1)

In [3]:
from tensorflow.keras.utils import to_categorical
X_train,X_val,X_test = np.array(X_train)/255.,np.array(X_val)/255.,np.array(X_test)/255.
y_train,y_val,y_test =  to_categorical((np.array(y_train)),9),to_categorical((np.array(y_val)),9),to_categorical((np.array(y_test)),9)

In [4]:
from tensorflow.keras import layers,models,initializers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
import sys
def model_build():
    size = 64
    factor=1e-5
    model = models.Sequential()
    model.add(layers.Conv2D(8, (5, 5), activation='relu', kernel_regularizer=l2(factor),input_shape=(size, size, 3)))
    model.add(layers.MaxPooling2D((2, 2)))


    model.add(layers.Conv2D(16, (5, 5), activation='relu',kernel_regularizer=l2(factor)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=l2(factor)))
    model.add(MaxPooling2D(pool_size=(2, 2)))


    model.add(layers.Conv2D(64, (3, 3), activation='relu',kernel_regularizer=l2(factor)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu',kernel_regularizer=l2(factor)))
    model.add(layers.Dense(128, activation='relu',kernel_regularizer=l2(factor)))
    
    model.add(layers.Dense(9, activation='softmax'))
    #model.summary()
    return model
from tensorflow.keras.regularizers import l2
model = model_build()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 8)         608       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 26, 26, 16)        3216      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        9280      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          3

In [ ]:
from tensorflow.python.keras.callbacks import TensorBoard
from tensorflow.keras.regularizers import l2
import datetime
#model = model_build(1e-5, 0.3)
model.compile(
   optimizer = 'adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#tsb = TensorBoard(log_dir='./logs')
history = model.fit(X_train, y_train,
          batch_size=128,
          epochs=20,
          validation_data=(X_val,y_val), callbacks=[tensorboard_callback])

Epoch 1/20
284/284 [==============================] - 14s 46ms/step - loss: 1.6266 - accuracy: 0.4326 - val_loss: 1.2351 - val_accuracy: 0.5615
Epoch 2/20
284/284 [==============================] - 13s 45ms/step - loss: 1.2300 - accuracy: 0.5586 - val_loss: 1.1380 - val_accuracy: 0.5924
Epoch 3/20
284/284 [==============================] - 13s 45ms/step - loss: 1.1497 - accuracy: 0.5886 - val_loss: 1.0978 - val_accuracy: 0.6082
Epoch 4/20
284/284 [==============================] - 13s 45ms/step - loss: 1.0902 - accuracy: 0.6127 - val_loss: 1.0399 - val_accuracy: 0.6263
Epoch 5/20
284/284 [==============================] - 13s 45ms/step - loss: 1.0386 - accuracy: 0.6294 - val_loss: 1.0020 - val_accuracy: 0.6369
Epoch 6/20
284/284 [==============================] - 13s 45ms/step - loss: 0.9816 - accuracy: 0.6481 - val_loss: 0.9510 - val_accuracy: 0.6587
Epoch 7/20
284/284 [==============================] - 13s 45ms/step - loss: 0.9468 - accuracy: 0.6616 - val_loss: 0.9091 - val_accuracy:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.epoch, history.history["accuracy"], label="accuracy")
plt.plot(history.epoch, history.history["loss"], label="loss")
plt.plot(history.epoch, history.history["val_accuracy"], label="val_accuracy")
plt.plot(history.epoch, history.history["val_loss"], label="val_loss")
plt.xlabel("epoch")
plt.legend()
plt.show()

In [ ]:
score = model.evaluate(X_test, y_test, verbose=0,batch_size=128)
print('Test loss:', score[0])
print('Test accuracy:', score[1])